## GraphDB에서 Cypher Query생성 샘플

In [6]:
from neo4j import GraphDatabase, basic_auth

BOLT_URL = "bolt://localhost:7687"
BOLT_PASSWORD = "your_password"


driver = GraphDatabase.driver(
  BOLT_URL,
  auth=basic_auth("neo4j", BOLT_PASSWORD))

cypher_query = '''
MATCH (movie:Movie {title:$favorite})<-[:ACTED_IN]-(actor)-[:ACTED_IN]->(rec:Movie)
 RETURN distinct rec.title as title LIMIT 20
'''

with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(cypher_query,
                      favorite="The Matrix").data())
  for record in results:
    print(record['title'])

driver.close()


The Matrix Reloaded
The Matrix Revolutions
The Devil's Advocate
The Replacements
Johnny Mnemonic
Something's Gotta Give
Cloud Atlas
V for Vendetta


/var/folders/pr/hngks2mn74d25jycvx6rxl400000gn/T/ipykernel_18534/1986060361.py:20: DeprecationWarning: read_transaction has been renamed to execute_read
  results = session.read_transaction(


In [16]:
from load_dotenv import load_dotenv
import os

load_dotenv()

from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import AzureChatOpenAI



graph = Neo4jGraph(url=BOLT_URL, username="neo4j", password=BOLT_PASSWORD, database="neo4j")

chain = GraphCypherQAChain.from_llm(
    AzureChatOpenAI(
        azure_endpoint=os.getenv("GRAPHRAG_BASE_URL"),
        openai_api_version=os.getenv("GRAPHRAG_API_VERSION"),
        azure_deployment=os.getenv("GRAPHRAG_LLM_DEPLOYMENT_NAME"),
        api_key=os.getenv("GRAPHRAG_API_KEY"), 
    ), 
    allow_dangerous_requests=True,
    graph=graph,
    verbose=True,
    return_intermediate_steps=True,
)


In [17]:
result=chain.invoke({"query": "Who played in Top Gun movie"})

print(f"Intermediate steps: {result['intermediate_steps']}")
print(f"Final answer: {result['result']}")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title: "Top Gun"})
RETURN p.name

Full Context:
[{'p.name': 'Tom Cruise'}, {'p.name': 'Kelly McGillis'}, {'p.name': 'Val Kilmer'}, {'p.name': 'Anthony Edwards'}, {'p.name': 'Tom Skerritt'}, {'p.name': 'Meg Ryan'}]

> Finished chain.
Intermediate steps: [{'query': 'cypher\nMATCH (p:Person)-[:ACTED_IN]->(m:Movie {title: "Top Gun"})\nRETURN p.name\n'}, {'context': [{'p.name': 'Tom Cruise'}, {'p.name': 'Kelly McGillis'}, {'p.name': 'Val Kilmer'}, {'p.name': 'Anthony Edwards'}, {'p.name': 'Tom Skerritt'}, {'p.name': 'Meg Ryan'}]}]
Final answer: Tom Cruise, Kelly McGillis, Val Kilmer, Anthony Edwards, Tom Skerritt, and Meg Ryan played in the Top Gun movie.


### Reference

https://python.langchain.com/docs/integrations/graphs/neo4j_cypher/